In [99]:
!pip -q install surprise

In [1]:
from requests_html import HTMLSession # for making request
import requests # for making request
import pandas as pd # for data processing
import numpy as np # for data processing
from tqdm import tqdm # for count time of iteration
import re
from tqdm import tqdm
from imdb import IMDb
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import hstack
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import cross_validate
from numpy.random import RandomState
from surprise import accuracy
from sklearn.neighbors import NearestNeighbors

# Problem

Prediction đóng một vai trò khá quan trọng trong nhiều **Recommendation Systems**.

Phần này sẽ là phần chính của đồ án, gồm 3 task prediction:
- Content Based Filtering: predict những bộ phim khác từ một bộ phim dựa trên những thuộc tính của nó như title, đạo diễn, diễn viên, thể loại, overview,... (dùng TF-IDF và CountVectorizer để vectorize, sau đó dùng cosine similarity matrix để dự đoán)
- Collaborative Filtering: predict điểm số mà user có thể muốn cho một phim nào đó (dùng SVD)
- Collaborative Filtering: predict những bộ phim cho một user dựa trên những bộ phim của những users có độ liên quan gần nhau (KNN)

Task predict đầu tiên sẽ phù hợp nếu chúng ta không biết user là ai.
2 task predict sau, bằng sự kết hợp lẫn nhau chọn ra top phim chung, sẽ phù hợp để recommend từ sở thích của user.

# Dự đoán phim dựa vào thuộc tính riêng

Ở bước này, ta thường thấy rằng một bộ phim có sự tương đồng với bộ phim khác nằm ở thể loại, kịch bản, cốt truyện, và thậm chí là diễn viên, đạo diễn.

Thế nên, mình sẽ lấy hết text từ các trường title, overview, genres từ file movie.csv, cùng toàn bộ các trường của file credit.csv, join hai file bằng movie_id

In [6]:
credit_df = pd.read_csv('../data/credit.csv', dtype={'id':str})
movie_df = pd.read_csv('../data/movie.csv', dtype={'id':str})

In [11]:
movie_credit_df = pd.merge(movie_df, credit_df, left_on='id', right_on='id', how='outer')

## Clean data bước 1:

Vì các field của credit đều có dạng list[dictionary] nên mình sẽ chuyển nó về dạng list[name], và lấy tối đa top 3 trong list đó

In [16]:
features = ['cast', 'directors', 'writers', 'producers', 'composers']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(literal_eval)

In [20]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [19]:
features = ['cast', 'directors', 'writers', 'producers', 'composers']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(get_list)

In [30]:
movie_credit_df[['cast', 'directors', 'writers', 'producers', 'composers', 'genres']].head(5)

,cast,directors,writers,producers,composers,genres
0,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",drama;romance
1,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",horror;thriller
2,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],biography;drama;sport
3,"[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],comedy;drama
4,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",family;music;musical;romance


## Clean data bước 2:

Mình sẽ chuyển tên các diễn viên về thành chữ thường và xoá khoảng trắng: ví dụ Johny Deep thành johnydeep. Điều này cũng giúp tránh phải hiện tượng trùng tên khác họ, ví dụ chữ Johny trong 2 cái tên khác nhau sẽ là 2 người khác nhau, nhưng nó có thể dẫn đến sự giống nhau khi vectorize, do đó phải tìm cách để nó có sự khác biệt, và xoá dấu cách như trên nhằm giải quyết vấn đề này

In [25]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [28]:
features = ['cast', 'directors', 'writers', 'producers', 'composers', 'genres']
for feature in features:
    movie_credit_df[feature] = movie_credit_df[feature].apply(clean_data)

In [29]:
movie_credit_df[['cast', 'directors', 'writers', 'producers', 'composers', 'genres']].head(5)

,cast,directors,writers,producers,composers,genres
0,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",drama;romance
1,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",horror;thriller
2,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],biography;drama;sport
3,"[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],comedy;drama
4,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",family;music;musical;romance


## Vectorize và predict function

Mình sẽ dùng TF-IDF để vectorize overview vì những keyword quan trọng của overview sẽ được score cao hơn khi vector, từ đó làm tăng độ đặc biệt cho mỗi nội dung.

Mình sẽ dùng CountVectorizer để vectorize những thông tin về credit, genre bởi vì mình cần những keyword giống nhau nhiều để thể hiện sự tương quan, nó hơi ngược với tư tưởng của TF-IDF và do đó TF-IDF không phù hợp.

Cuối cùng mình concat 2 ma trận vector lại với nhau, từ đó tính cosine similarity và chọn top.

In [61]:
def create_soup(x):
    ans = ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['directors']) + ' ' + ' '.join(x['genres'].split(";")) + ' ' + ' '.join(x['writers']) + ' ' + ' '.join(x['producers']) + ' ' + ' '.join(x['composers'])
    return ans

In [62]:
movie_credit_df['soup'] = movie_credit_df.apply(create_soup, axis=1)

In [63]:
movie_credit_df.head(5)

,level_0,index,id,title,runtimes,genres,vote_counts,average_rating,overview,cast,directors,writers,producers,composers,soup
0,0,0,0337578,Gardener,181,drama;romance,15084,7.4,Raj Malhotra and wife Pooja have four sons. Th...,"[amitabhbachchan, hemamalini, salmankhan]",[ravichopra],"[shafiqansari, satishbhatnagar, b.r.chopra]","[ashwanichopra, b.r.chopra]","[aadeshshrivastava, uttamsingh]",amitabhbachchan hemamalini salmankhan ravicho...
1,1,1,0113253,Halloween: The Curse of Michael Myers,87,horror;thriller,31301,4.8,Six years after Michael Myers' last massacre i...,"[donaldpleasence, paulrudd, mariannehagan]",[joechappelle],"[debrahill, johncarpenter, danielfarrands]","[malekakkad, moustaphaakkad, paulfreeman]","[alanhowarth, paulrabjohns]",donaldpleasence paulrudd mariannehagan joecha...
2,2,2,0995868,Pele: Birth of a Legend,107,biography;drama;sport,16402,7.2,Pele's meteoric rise from the slums of Sao Pau...,"[kevindepaula, leonardolimacarvalho, seujorge]","[jeffzimbalist, michaelzimbalist]","[jeffzimbalist, michaelzimbalist]","[alexandredauman, guyeast, caíquemartinsferreira]",[a.r.rahman],kevindepaula leonardolimacarvalho seujorge je...
3,3,3,4901306,Perfect Strangers,96,comedy;drama,56387,7.8,"On a warm summer evening, the loving couple of...","[giuseppebattiston, annafoglietta, marcogiallini]",[paologenovese],"[filippobologna, paolocostella, paologenovese]","[marcobelardi, ughettacurto, marcogiannoni]",[mauriziofilardo],giuseppebattiston annafoglietta marcogiallini...
4,4,4,0361696,Raise Your Voice,107,family;music;musical;romance,26730,5.9,This film is about a teenage girl who is very ...,"[hilaryduff, oliverjames, davidkeith]",[seanmcnamara],"[mitchrotter, samschreiber]","[davidbrookwell, a.j.dix, tobyemmerich]","[machinehead, aaronzigman]",hilaryduff oliverjames davidkeith seanmcnamar...


In [76]:
count = CountVectorizer(stop_words='english')
count_matrix_1 = count.fit_transform(movie_credit_df['soup'])
tfidf = TfidfVectorizer(stop_words='english')
count_matrix_2 = tfidf.fit_transform(movie_credit_df['overview'])

In [91]:
count_matrix = hstack((count_matrix_1, count_matrix_2))

In [93]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [94]:
def get_recommendations(title, cosine_sim=cosine_sim, top=10):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:top+1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movie_credit_df['title'].iloc[movie_indices]

In [ ]:
movie_credit_df = movie_credit_df.reset_index()
indices = pd.Series(movie_credit_df.index, index=movie_credit_df['title'])

## Thế là xong, mình sẽ thử recommend theo vài bộ phim

Có thể thấy, ở recommend đầu tiên, những bộ phim nói về chủ đề người dơi sẽ được recommend nhiều, cũng như các bộ phim có nội dung thuộc về hành động, viễn tưởng giống với phim đã chọn sẽ vào top similar.

Ở recommend thứ hai, rõ ràng những bộ phim nằm trong vũ trụ Marvel đều lọt top khá cao nhờ vào sự giống nhau không chỉ nội dung mà còn diễn viên, đạo diễn, ...

In [96]:
get_recommendations('The Dark Knight Rises', cosine_sim, top=20)

7725                       The Dark Knight
3373                         Batman Begins
3508                          Interstellar
2150                             Inception
2538                          The Prestige
3986                             Following
798                                Dunkirk
8564                          Man of Steel
4118                                 Tenet
429                                Memento
1696                               Ghajini
6120                          Broken Arrow
122                    Clash of the Titans
163                            King Arthur
1837                   X-Men: Dark Phoenix
3667                Exodus: Gods and Kings
6932                   Wrath of the Titans
419                      National Treasure
2619    Sherlock Holmes: A Game of Shadows
2814                       Sherlock Holmes
Name: title, dtype: object

In [97]:
get_recommendations('Avengers: Age of Ultron', cosine_sim, top = 20)

2446                          The Avengers
4090                Avengers: Infinity War
2584                            Iron Man 3
631                      Avengers: Endgame
6743                              Serenity
6758                            Iron Man 2
2965            Captain America: Civil War
6673                Spider-Man: Homecoming
2285                                  Hulk
238                               Iron Man
58                    Thor: The Dark World
4599    Captain America: The First Avenger
1297                            Spider-Man
520                           Spider-Man 2
8072             Spider-Man: Far from Home
3928           Men in Black: International
4441                  Ant-Man and the Wasp
4426               Guardians of the Galaxy
2805                        Thor: Ragnarok
8131                         Black Panther
Name: title, dtype: object

# Dự đoán preference của user đối với phim từ bảng rating của mỗi user với một số phim

Mình sẽ dùng một kĩ thuật gọi là SVD (Single Value Decomposition), với metric đánh giá là Root Mean Square Error (RMSE). Giá trị của RMSE càng thấp gần về 0 thì performance càng tốt.

In [82]:
ratings_df = pd.read_csv('../data/rating.csv', dtype={'movie_id':str, 'user_id':str})
ratings_df.head()

,movie_id,user_id,rating
0,4799050,0872211,1
1,4799050,12800375,1
2,4799050,45461313,1
3,4799050,29907479,1
4,4799050,0179626,1


Mình sẽ shuffle dataframe này để giá trị rating sẽ không còn thứ tự nữa

In [83]:
ratings_df = ratings_df.sample(frac=1).reset_index(drop=True)

Vì thư viện mô hình predict chỉ cho ra kết quả từ 1 đến 5, nên mình sẽ scale thang rating từ 10 thành 5

In [84]:
ratings_df['rating'] /= 2

Chia data thành 2 tập với tỉ lệ 7:3. Tuy nhiên để có thể predict được cho mọi user, mình phải lấy ra ít nhất một row cho mỗi user_id trước, rồi lấy sample sau, cuối cùng sẽ loại bỏ duplicate.

In [ ]:
rng = RandomState()

train_list = []
for id in tqdm(ratings_df['user_id'].unique()):
    train_list.append(ratings_df[ratings_df['user_id'] == id].iloc[[0]])

train_list.append(ratings_df.sample(frac=0.7, random_state=rng))
train_df = pd.concat(train_list, ignore_index = True)
train_df.drop_duplicates(keep = 'first', inplace = True)
test_df = pd.concat([train_df, ratings_df], ignore_index = True).drop_duplicates(keep=False)

Đánh giá thử tập train

In [265]:
reader = Reader()
rating_train_data = Dataset.load_from_df(train_df[['user_id', 'movie_id', 'rating']], reader)
rating_test_data = Dataset.load_from_df(test_df[['user_id', 'movie_id', 'rating']], reader)

In [266]:
algo = SVD()

In [267]:
cross_validate(algo, rating_train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2458  1.2492  1.2487  1.2444  1.2496  1.2475  0.0021  
MAE (testset)     1.0029  1.0074  1.0056  1.0018  1.0064  1.0048  0.0021  
Fit time          35.04   34.80   35.36   35.21   35.80   35.24   0.34    
Test time         1.32    1.19    1.28    1.10    3.31    1.64    0.84    


{'test_rmse': array([1.24576077, 1.24919091, 1.24874247, 1.24435894, 1.2495603 ]),
 'test_mae': array([1.00292287, 1.00740319, 1.00563021, 1.00183054, 1.00644815]),
 'fit_time': (35.043339014053345,
  34.79702591896057,
  35.35840559005737,
  35.207868576049805,
  35.80226492881775),
 'test_time': (1.320204734802246,
  1.1858248710632324,
  1.2785828113555908,
  1.1040306091308594,
  3.3101487159729004)}

Mean của RMSE và MAE khá thấp, chúng ta có thể hi vọng nó đủ tốt để dự đoán. Bây giờ sẽ đến bước train

In [268]:
train_set = rating_train_data.build_full_trainset()
algo.fit(train_set)

Mình sẽ test từ tập `test_df` bằng metric RMSE để xem kết quả có thấp như cross_validate ở tập train hay không

In [291]:
RMSE = 0
for i in tqdm(range(test_df.shape[0])):
    movie_id = int(test_df.iloc[i]['movie_id'])
    user_id = int(test_df.iloc[i]['user_id'])
    rating = test_df.iloc[i]['rating']
    prediction = algo.predict(user_id, movie_id, rating)
    RMSE += (prediction[2] - prediction[3])**2

RMSE /= test_df.shape[0]

print(RMSE)

100%|████████████████████████████████████████████████████████████████████████| 264356/264356 [01:29<00:00, 2953.08it/s]

1.1869374887463162


Độ lỗi ở tập test khá thấp hơn một chút so với tập train.

Mình sẽ xem thử một vài kết quả, với r_ui là rating thật, còn est là rating dự đoán

In [286]:
sometest_df = test_df.head(20)

In [287]:
for i in range(sometest_df.shape[0]):
    movie_id = int(sometest_df.iloc[i]['movie_id'])
    user_id = int(sometest_df.iloc[i]['user_id'])
    rating = sometest_df.iloc[i]['rating']
    prediction = algo.predict(user_id, movie_id, rating)
    print(prediction)

user: 6663090    item: 55824      r_ui = 4.50   est = 4.77   {'was_impossible': False}
user: 4248714    item: 974661     r_ui = 1.50   est = 2.59   {'was_impossible': False}
user: 438066     item: 120780     r_ui = 3.50   est = 3.54   {'was_impossible': False}
user: 438066     item: 416320     r_ui = 4.00   est = 3.73   {'was_impossible': False}
user: 7813355    item: 99487      r_ui = 3.50   est = 4.28   {'was_impossible': False}
user: 4248714    item: 367882     r_ui = 3.50   est = 2.64   {'was_impossible': False}
user: 56005872   item: 2582802    r_ui = 4.50   est = 4.04   {'was_impossible': False}
user: 23055365   item: 1014759    r_ui = 2.00   est = 3.47   {'was_impossible': False}
user: 20552756   item: 76054      r_ui = 3.50   est = 3.50   {'was_impossible': False}
user: 2488512    item: 89173      r_ui = 2.00   est = 1.99   {'was_impossible': False}
user: 1234929    item: 1356864    r_ui = 3.00   est = 3.42   {'was_impossible': False}
user: 2898520    item: 117331     r_ui = 3.

# Dự đoán những users gần "giống" với user đã chọn từ genre, từ đó chọn ra những bộ phim user này chưa xem nhưng những neighbor users đã xem rồi

In [22]:
movie_df = pd.read_csv('../data/movie.csv', dtype={'id':str})

Đưa genre từ data dạng string thành one-hot vector

In [23]:
extract_genre_df = movie_df['genres'].str.get_dummies(';')

In [24]:
movie_df = pd.concat([movie_df[['id', 'title']], extract_genre_df], axis = 1)

Merge table có user và movie lại

In [25]:
movie_user_df = pd.merge(movie_df, ratings_df, left_on='id', right_on='movie_id', how='outer')

In [28]:
movie_user_df.head(10)

,id,title,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,...,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,movie_id,user_id,rating
0,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,91679541,5.0
1,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,1341621,5.0
2,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,11365802,0.5
3,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,19440893,5.0
4,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,5842500,4.5
5,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,47571953,1.5
6,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,19067761,0.5
7,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,3763414,5.0
8,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,0375636,1.0
9,0337578,Gardener,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0337578,24774941,4.5


Đưa data về dạng user và tổng số thể loại đã xem (qua phim)

In [26]:
new_movie_user_df = movie_user_df.drop(["movie_id","title", "rating"],axis=1)

In [27]:
new_movie_user_df.head(10)

,id,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,user_id
0,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,91679541
1,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1341621
2,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,11365802
3,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,19440893
4,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,5842500
5,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,47571953
6,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,19067761
7,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,3763414
8,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0375636
9,0337578,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,24774941


In [15]:
users_moviemat = movie_user_df.groupby("user_id").sum()
X = users_moviemat.iloc[:,:].values
users_moviemat

,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
user_id,,,,,,,,,,,,,,,,,,,,,
0000002,0,0,0,0,3,1,1,0,0,0,...,1,0,0,1,0,1,0,1,0,0
0000005,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000006,5,1,0,0,2,5,10,0,0,0,...,0,0,5,0,7,2,1,4,1,0
0000011,1,1,0,1,4,1,9,0,1,0,...,1,0,0,1,5,2,0,2,0,0
0000015,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992944,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
99995655,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9999610,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
index = users_moviemat.index
dict_index_user = {}
dict_index_X = {}

for i in tqdm(range(len(index))):
    dict_index_user[str(index[i])] = i
    dict_index_X[i] = str(index[i])

100%|██████████████████████████████████████████████████████████████████████| 287948/287948 [00:00<00:00, 471730.70it/s]


In [68]:
dict_index_X[2]

'0000006'

Dùng KNN để tìm những "users gần nhất"

In [63]:
classifier = NearestNeighbors()
classifier.fit(X)

NearestNeighbors()

In [71]:
convert_uid = None
while convert_uid is None:
    uid = input("Enter User Id: ")
    convert_uid = dict_index_user.get(uid, None)
li = classifier.kneighbors([X[convert_uid]], n_neighbors=5, return_distance=False)
ans = []
for i in range(len(li[0])):
    ans.append(str(dict_index_X[li[0][i]]))

Enter User Id: 0000006


In [72]:
ans

['0000006', '4509599', '3278701', '1680051', '17823096']

In [75]:
current_user = movie_user_df.loc[movie_user_df["user_id"]==ans[0],:]["title"].values
movies_list = []

for id in ans[1:]:
    similar_user = movie_user_df.loc[movie_user_df["user_id"]==id,:]["title"].values
    movie = [movie for movie in similar_user if movie not in current_user]
    
    movies_list.extend(movie)

In [87]:
movies_list[:10]

['Bhagam Bhag',
 'Jaane Tu',
 'Kabhi Haan Kabhi Naa',
 'Watch Out Ladies',
 'Don',
 'Omkara',
 'Slave',
 'Iqbal',
 'Kabhi Khushi Kabhie Gham...',
 'Pardes']

Có thể thấy những bộ phim được gợi ý là phim Ấn Độ, user đã chọn cũng đã xem phim Ấn Độ Gardener